# Interactive PSF photometry using Simulated NIRCam Image

This is the interactive version of the accompanying `NIRCAM_PSF_phot_mpl.ipynb` by Matteo Correnti. This notebook was contributed by Pey Lian Lim, with inputs from Matteo Correnti and Erik Tollerud. Jupyter Lab is recommended because you can pop out the displays into separate tabs as needed.

Its goal is to perform PSF photometry on simulated JWST NIRCam image for a "crowded" field based on use case laid out in https://outerspace.stsci.edu/display/JWSTDATF/JWST+Data+Analysis+Tool+Use+Case+for+crowded+field+imaging .

Workflow:

* Use `photutils` to find stars in a given image.
* Circle the stars on the image.
* For each star found, make a cutout image.
* Display the cutouts for selection.
* Select a subset of the stars based on cutouts.
* Use the selection to build a PSF.
* Subtract PSF from image (a sub-image is used here to shorten run time).
* Display residual image and compare it to original sub-image.
* User may repeat the steps above until PSF is deemed satisfactory.
* Save out the final selection to a table file.

Future work:

* Repeat for the same field in a different filter.
* Use photometry results from both filters to build a CMD.

The interactive portion was done using `astrowidgets` with Ginga backend. See https://astrowidgets.readthedocs.io for additional details about the widget, including installation notes.

In [1]:
from astrowidgets import ImageWidget as _ImageWidget

We subclass `ImageWidget` here to add ASDF loader. In the future, the loader method would be in `astrowidgets` and this subclass would no longer be necessary.

# NOTE

The ASDF portion of this work was not successful during the Hack Day because reading JWST ASDF file could not be done on Windows. See https://github.com/spacetelescope/jwst/issues/3132 for more details. Theoretically, this would work on Linux or OSX, but currently untested.

In [2]:
# Reading ASDF in Ginga needs this to be specified early on
from ginga.util import wcsmod

wcsmod.use('astropy_ape14')

True

In [180]:
# TODO: Do we need to port this upstream to astrowidgets?
class ImageWidget(_ImageWidget):
    
    _other_viewer = None  # Hacky hack hack
    
    # TODO: Need to generalize this method before moving it
    #       to astrowidgets package.
    def load_jwst_asdf(self, filename):
        """Load ASDF extension from JWST Level 2 data.

        .. note::

            This needs to use ``'astropy_ape14'`` WCS module in Ginga.
            This also currently needs ``jwst`` pipeline to be installed.

        Parameters
        ----------
        filename : str
            Filename of JWST Level 2 file.
            It is a FITS file with ASDF-in-FITS extension.

        """
        import asdf

        image = AstroImage(logger=self.logger)

        with asdf.open(filename) as asdf_f:
            asdf_f['wcs'] = asdf_f['meta']['wcs']
            image.load_asdf(asdf_f, data_key='data')

        self._viewer.set_image(image)
        
    def set_color_map(self, cmap):
        """Set colormap to the given colormap name.
        
        cmap : str
            Colormap name.
            
        """
        self._viewer.set_color_map(cmap)
        
    # TODO: A hack until Matt Craig can get back to me about traitlets
    def link_viewer(self, other_viewer):
        self._other_viewer = other_viewer
        
    # Hacky hack hack
    def _mouse_click_cb(self, viewer, event, data_x, data_y):
        if isinstance(self._other_viewer, ImageWidget):
            self._other_viewer._mouse_click_cb(self._other_viewer._viewer, event, data_x, data_y)
            
        super()._mouse_click_cb(self._viewer, event, data_x, data_y)

With the ASDF stuff out of the way, we use the widget as intended now.

In [181]:
from ginga.misc.log import get_logger

logger = get_logger('my viewer', log_stderr=True, log_file=None, level=30)

In [182]:
w = ImageWidget(logger=logger)

Input file can be downloaded from https://stsci.app.box.com/s/1jp2g3uau0cgo0eq9uan3clr5rxjytt9 . Ask Matteo Correnti for permission.

In [183]:
filename = 'V1069002001P000000000110n_A2_F200W_cal.fits'

# NOTE: This does not work without JWST pipeline installed
#       and pipeline cannot install on Windows.
# https://github.com/spacetelescope/jwst/issues/3132
# Loads the ASDF portion of the JWST Level 2 data.
#w.load_jwst_asdf(filename)

# Loads FITS portion of JWST Level 2 data.
from astropy.nddata import CCDData
numhdu = 1
ccd = CCDData.read(filename, hdu=numhdu, format='fits', unit='electron/s')
w.load_nddata(ccd)

INFO: using the unit electron/s passed to the FITS reader instead of the unit DN/s in the FITS file. [astropy.nddata.ccddata]


A viewer will be shown after running the next cell.
In Jupyter Lab, you can split it out into a separate view by right-clicking on the viewer and then select
"Create New View for Output". Then, you can drag the new
"Output View" tab, say, to the right side of the workspace. Both viewers are connected to the same events.

**NOTE:** There is a "jumping" bug when you mouse over the image display. It has been reported at https://github.com/mwcraig/ipyevents/issues/37

In [184]:
w

ImageWidget(children=(Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\x…

This next cell captures print outputs. You can pop it out like the viewer above. It is very convenient for debugging purpose.

In [185]:
# Capture print outputs from the widget
display(w.print_out)

Output()

In [186]:
# This reminds me of the stretch and cuts options for visualization.
print(w.stretch_options)
print(w.autocut_options)

['linear', 'log', 'power', 'sqrt', 'squared', 'asinh', 'sinh', 'histeq']
('minmax', 'median', 'histogram', 'stddev', 'zscale')


In [187]:
# Change stretch, cut levels, and colormap.
# Also see https://github.com/astropy/astrowidgets/issues/63
w.stretch = 'sqrt'
w.cuts = (0, 10)
w.set_color_map('gray_r')

In [14]:
import warnings
from photutils import find_peaks
from photutils.centroids import centroid_2dg

# Find the stars in the whole image.
data = ccd.data
with warnings.catch_warnings():
    warnings.simplefilter('ignore')  # Ignore warning about fitting
    peaks_tbl = find_peaks(data, threshold=75, box_size=75, centroid_func=centroid_2dg)
peaks_tbl['peak_value'].info.format = '%.8g'

print(len(peaks_tbl))
print(peaks_tbl[:10])

349
x_peak y_peak peak_value     x_centroid          y_centroid     
------ ------ ---------- ------------------ --------------------
   665      4  361.93878   666.055243094477 -0.21515474934364764
  1575      6  86.846886 1575.6256359372442    6.783013163073283
  1831     14  211.34196 1830.9447410770383   14.404077728039052
   398     17   110.0424  398.2843689338144    16.72563423949063
  1325     18  96.866768   1325.21577828733   17.989053097051368
  1652     25  76.919403 1650.9359909764003   60.914631483893785
  1067     36    138.356  1066.988490517874   35.724792017163004
  1213     42  439.62793  1213.260368681354   41.997141666629375
   906     44   196.6805  906.2420413142759    43.73217206503919
  1930     67  280.50177 1930.0050386886544    67.44105477601812


In [40]:
# List Circle parameters for Ginga backend.
# This is useful if you want to see how you can customize the circles.
from ginga.canvas.types.basic import Circle

circle_params = Circle.get_params_metadata()  # list of dict
print('Param names:', [p['name'] for p in circle_params])

key = 'linestyle'  # Replace with your param of interest
print([p for p in circle_params if p['name'] == key][0])

Param names: ['coord', 'x', 'y', 'radius', 'linewidth', 'linestyle', 'color', 'alpha', 'fill', 'fillcolor', 'fillalpha', 'showcap']
{'name': 'linestyle', 'type': <class 'str'>, 'default': 'solid', 'valid': ['solid', 'dash'], 'description': 'Style of outline (default solid)'}


**Future Work:** Click to select/deselect markers. Also show how to "derive parameters" (e.g., FWHM?) for a selected marker.

In [82]:
# Set the marker parameters.
# The parameters can be discovered using the previous cell.
w.marker = {'color': 'blue', 'radius': 20, 'type': 'circle', 'alpha': 0.5, 'linewidth': 1.5}

# Show the stars found on the image.
w.add_markers(peaks_tbl, x_colname='x_peak', y_colname='y_peak')

In [21]:
from astropy.stats import sigma_clipped_stats

# Calculate background
mean_val, median_val, std_val = sigma_clipped_stats(data, sigma=2., maxiters=None)

In [22]:
# Simple background subtraction
data_no_bg = data - median_val

In [23]:
from astropy.nddata import NDData

# Create NDData container to store data without background for the next step
nddata = NDData(data=data_no_bg)

In [24]:
from astropy.table import Table

# This is required by photutils, which is weird.
# Opened issue at https://github.com/astropy/photutils/issues/800
stars_tbl = Table()
stars_tbl['x'] = peaks_tbl['x_peak']
stars_tbl['y'] = peaks_tbl['y_peak']

In [26]:
from photutils.psf import extract_stars

# Create 25x25 pixels cutout of found peaks.
# NOTE: Stars too close to edge will be excluded with a warning, which we ignore.
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    stars = extract_stars(nddata, stars_tbl, size=25)

Create a new interactive display for cutout.

In [188]:
w_cutout = ImageWidget(logger=logger)
w_cutout

ImageWidget(children=(Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\x…

In [190]:
w_cutout.stretch = 'linear'
w_cutout.cuts = 'zscale'
w_cutout.set_color_map('viridis')

In [44]:
# These have to be global variables for button widgets below.
i_star = 0
cur_star = None

In [45]:
# Function to display a given cutout by index
def display_cutout():
    global cur_star
    cur_star = stars.all_good_stars[i_star]
    w_cutout.load_array(cur_star.data)
    
    # Print in the cell that captures widget print-out above
    with w.print_out:
        print('Cutout', cur_star.id_label, cur_star.center)

In [46]:
# Display the first cutout
display_cutout()

In [47]:
# Create empty list to store selected stars
selected_stars = []

Next, we create the buttons to enable interative selection of the stars based on displayed cutout. "Next" button cycles to the next cutout in the list, while "Select" adds the current cutout to `selected_stars`. Clicking "Next" without clicking "Select" first effectively excludes the displayed cutout from `selected_stars`.

**Future work:** Enable starting the selection process at a given point in `stars.all_good_stars`. For this to work in a meaningful way, there also needs to be a way to save the state of `selected_stars` and load it back next time user re-runs the notebook.

In [48]:
import ipywidgets
from IPython.display import display

next_btn = ipywidgets.widgets.Button(description="Next")
sel_star_btn = ipywidgets.widgets.Button(description="Select")
display(ipywidgets.HBox([next_btn, sel_star_btn]))


def next_star_cb(b):
    """Display next star in the cutout list."""
    global i_star
    i_star += 1
    display_cutout()
    
    
def select_star_cb(b):
    """Add the displayed star to the selected list.
    This will stay at the displayed star until Next is clicked."""
    global selected_stars
    selected_stars.append(cur_star)
    
    
next_btn.on_click(next_star_cb)
sel_star_btn.on_click(select_star_cb)

Before running this next cell, make sure `selected_stars` are done being populated using the interactive selection tool above.

In [49]:
from photutils.psf.epsf_stars import EPSFStars
selected_stars = EPSFStars(selected_stars)

print('Selected {} stars'.format(len(selected_stars)))
print('First 10 selected stars:')
for s in selected_stars[:10]:
    print(s.id_label, s.center)

Selected 28 stars
First 10 selected stars:
4 [398.  17.]
5 [1325.   18.]
7 [1067.   36.]
9 [906.  44.]
10 [1930.   67.]
11 [520.  72.]
12 [817.  73.]
18 [1270.   89.]
20 [116. 106.]
21 [304. 130.]


In [50]:
from photutils import EPSFBuilder

# Build the PSF
epsf_builder = EPSFBuilder(oversampling=4, maxiters=3,
                           progress_bar=False)
epsf, fitted_stars = epsf_builder(selected_stars)

We now display the resulting PSF in the same viewer we used for cutout.

In [86]:
w_cutout.load_array(epsf.data)

Now, perform PSF photometry if the PSF above is satisfactory. If not, repeat the cutout selection step.

In [52]:
from photutils.detection import DAOStarFinder, IRAFStarFinder
from photutils.psf import DAOGroup
from photutils.psf import IntegratedGaussianPRF
from photutils.background import MMMBackground
from photutils.background import MADStdBackgroundRMS
from astropy.modeling.fitting import LevMarLSQFitter
from astropy.stats import gaussian_sigma_to_fwhm

sigma_psf = 1.25

bkgrms = MADStdBackgroundRMS()

std = bkgrms(data)

iraffind = IRAFStarFinder(threshold=10*std,
                          fwhm=sigma_psf*gaussian_sigma_to_fwhm,
                          minsep_fwhm=0.01, roundhi=1.0, roundlo=-1.0,
                          sharplo=0.30, sharphi=1.40)

daogroup = DAOGroup(2.0 * sigma_psf * gaussian_sigma_to_fwhm)

mmm_bkg = MMMBackground()

psf_model = epsf.copy()

fitter = LevMarLSQFitter()

In [53]:
# Use a sub-image here to speed things up during prototyping.
# In real world, whole image would be used.
data1 = data[0:200, 0:200]

In [57]:
from photutils.psf import IterativelySubtractedPSFPhotometry

with warnings.catch_warnings():
    warnings.simplefilter('ignore')  # Ignore fitting warnings
    photometry = IterativelySubtractedPSFPhotometry(
        finder=iraffind, group_maker=daogroup,
        bkg_estimator=mmm_bkg, psf_model=psf_model,
        fitter=LevMarLSQFitter(),
        niters=2, fitshape=(11, 11), aperture_radius=5)
    result_tab = photometry(data1)
    residual_image = photometry.get_residual_image()

In [196]:
import numpy as np
from astropy.stats import biweight_location

# Print some stats from residual image.
print('Median residual: {:.4f}'.format(np.median(residual_image)))
print('Biweight location: {:.4f}'.format(biweight_location(residual_image)))

Median residual: -0.0038
Biweight location: -0.0020


Now, we re-purpose the display that was displaying the whole image to display `data1` and the one that was displaying cutout to display its residual after PSF subtraction.

In [87]:
# Un-circle the stars found.
w.reset_markers()

In [189]:
w.load_array(data1)
w_cutout.load_array(residual_image)

Now we "lock" the two displays at pixel level. The way this is currently done, you have to click on the viewer of `data1` so it pans to the pixel you clicked on. The viewer for `residual_image` will automatically follow.

**TODO:** Need to link them properly using `traitlets`. Will need to consult Matt Craig about this. Want to link on zoom and pan.

**Future Work:** Add a third viewer to display uncertainties and link it the same way.

In [193]:
w.link_viewer(w_cutout)
w.click_center = True
w_cutout.click_center = True

If the residual image is not satisfactory, repeat the analysis steps above as needed.

In [197]:
# PSF photometry result using sub-image.
result_tab.sort('id')
result_tab

x_0,x_fit,y_0,y_fit,flux_0,flux_fit,id,group_id,flux_unc,x_0_unc,y_0_unc,iter_detected
float64,float64,float64,float64,float64,float64,int32,int32,float64,float64,float64,int32
41.9958553565839,41.7915352391071,4.33512111445422,4.422016749583286,3.9495902382608,123.78431030003614,1,1,9.98918372386569,0.11266952675741267,0.09704003036088321,1
79.0900593026507,77.43278817969596,4.663149121765862,2.5965934629220038,-7.889413969405278,-103.50157679114186,1,1,31.569991288316377,0.2511500401227103,0.2658074440725549,2
101.34167438524037,97.38984155153547,26.809216327617822,35.593678517332215,-16.481938575021584,-1449.1655843802982,2,2,118.57041780067934,118.57041780067934,118.57041780067934,2
74.60089299488602,74.39931066228506,4.250560972956118,4.4644971271171485,59.9048150716688,336.9604560735015,2,2,123.78431030003614,123.78431030003614,123.78431030003614,1
79.47439950199403,79.91537442048092,4.3416843115536015,4.708442939739389,64.02527322180956,409.8357942462628,3,2,41.7915352391071,41.7915352391071,41.7915352391071,1
97.67307621096161,118.57041780067934,28.01869933222206,27.47805022962194,-19.13305166917766,137.45999841958428,3,2,137.45999841958428,137.45999841958428,137.45999841958428,2
111.19987676335725,111.28223985549357,4.295625666317926,4.366394905877891,26.733915725228496,626.9667684816151,4,3,32.06694381625996,0.0773041678537691,0.0656818659989552,1
34.48571177751445,35.515730298474026,46.83396308549639,49.11116458942104,-11.055857388272383,-56.34911656604048,4,3,27.94375412836814,0.5542108872073624,0.7255204476519331,2
77.20640000654318,77.21887649460652,4.984939990560829,5.087390567902911,73.96969210701036,794.4296644753246,5,2,4.422016749583286,4.422016749583286,4.422016749583286,1


In [198]:
print(result_tab.colnames)

['x_0', 'x_fit', 'y_0', 'y_fit', 'flux_0', 'flux_fit', 'id', 'group_id', 'flux_unc', 'x_0_unc', 'y_0_unc', 'iter_detected']


**Future Work:** Repeat this for a second filter of the same field. Use photometry results from both filters to build a color-magnitude diagram (CMD).